# Steady state solution

In [13]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

include("../constants.jl")
include("../get_mesh_data.jl")
include("../process.jl")
include("../definitions/source.jl")
include("../definitions/linear_reluctivity.jl")
include("../definitions/assemble_Kf.jl")

const MESH_LOCATION = "../../mesh/transformer_stedin.msh"
const OUTPUT_LOCATION = "./out/"

"./out/"

In [14]:
gmsh.open(MESH_LOCATION)
mshdata = get_mesh_data();

Info    : Reading '../../mesh/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../mesh/transformer_stedin.msh'


## Problem definition

Find $A_z$ in the system
$$ -\nabla \times \left[\frac{1}{\mu}\nabla A_z\right] = \mathbf J_0, $$
where
- $A_z$ is the current density in the $z$ direction.
- $\mu$ is the permeability of the core.
- $\mathbf J_0$ is the imposed source current density.
- $\sigma$: the conductivity of the material (not used).

### Parameters

In [15]:
"Primary peak phase current"
Ip = 0;

"Secondary peak phase current"
Is = 777.62;

Np = 266;
Ns = 6;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Vacuum permeability
mu0 = 4e-7 * pi;

# Relative permeability of the core
mur = 1000;    

### Source, linear reluctivity per element (no conductivity due to steady state)


In [16]:
group_id_per_element = mshdata.e_group
source_per_element = map(source, group_id_per_element);

# Relative permeability model
reluctivity_per_element = map(linear_reluctivity, group_id_per_element);

zero_conductivity(group_id) = 0
conductivity_per_element = map(zero_conductivity, group_id_per_element);

## System of equations

In [17]:
K, f = assemble_Kf(mshdata, source_per_element, reluctivity_per_element);

How long does this take?

In [18]:
@timev K, f = assemble_Kf(mshdata, source_per_element, reluctivity_per_element);

  0.863944 seconds (1.67 M allocations: 1.702 GiB, 15.04% gc time)
elapsed time (ns):  863943700
gc time (ns):       129903500
bytes allocated:    1827252304
pool allocs:        1671173
non-pool GC allocs: 0
malloc() calls:     5
free() calls:       3
minor collections:  0
full collections:   1


Solve the system.

In [ ]:
u = K \ f;

## Post

In [ ]:
# Post-process for magnetic field and current density
omega = 0
B, H, Wm, Jel = process(mshdata, u, source_per_element, reluctivity_per_element, conductivity_per_element, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

Save the result

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid(string(OUTPUT_LOCATION, "transformer1.vtu"), points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
print("Saving result in a file...")
outfiles = vtk_save(vtkfile);
println(" Done.")